In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import datasets, transforms
from torchvision.utils import save_image

# Define constants
IMG_SIZE = 512  # Set to 512x512 for the new output size
LATENT_DIM = 200  # Updated latent dimension
BATCH_SIZE = 16  # Updated batch size
EPOCHS = 100

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.init_size = IMG_SIZE // 32  # Adjusted for 512x512 images (512 / 32 = 16)
        self.l1 = nn.Sequential(nn.Linear(LATENT_DIM, 512 * self.init_size ** 2))
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(512),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(512, 256, 3, stride=1, padding=1),
            nn.BatchNorm2d(256, 0.8),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(256, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(64, 32, 3, stride=1, padding=1),
            nn.BatchNorm2d(32, 0.8),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(32, 16, 3, stride=1, padding=1),
            nn.BatchNorm2d(16, 0.8),
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 3, 3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 512, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 4, stride=2, padding=1),   # 512x512 -> 256x256
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),  # 256x256 -> 128x128
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, stride=2, padding=1), # 128x128 -> 64x64
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, stride=2, padding=1), # 64x64 -> 32x32
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1024, 4, stride=2, padding=1), # 32x32 -> 16x16
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(1024, 2048, 4, stride=2, padding=1), # 16x16 -> 8x8
            nn.BatchNorm2d(2048),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Flatten(),
            nn.Linear(2048 * 8 * 8, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        validity = self.model(img)
        return validity


transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

dataloader = DataLoader(
    ConcatDataset([datasets.Flowers102(root='../../data/flowers', split='train', download=True, transform=transform),
                   datasets.Flowers102(root='../../data/flowers', split='val', download=True, transform=transform),
                   datasets.Flowers102(root='../../data/flowers', split='test', download=True, transform=transform)]),
    batch_size=BATCH_SIZE, shuffle=True
)

# Initialize models
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss function
adversarial_loss = nn.BCELoss()

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Training
for epoch in range(EPOCHS):
    for i, (imgs, _) in enumerate(dataloader):

        # Configure input
        real_imgs = imgs.to(device)
        batch_size = real_imgs.size(0)

        # Adversarial ground truths
        valid = torch.ones(batch_size, 1, requires_grad=False).to(device)
        fake = torch.zeros(batch_size, 1, requires_grad=False).to(device)

        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = torch.randn(batch_size, LATENT_DIM).to(device)

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss for real images
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        # Loss for fake images
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        # Total discriminator loss
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()

        # Loss for fake images with flipped labels
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        print(f"[Epoch {epoch}/{EPOCHS}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

    # Save sample images
    if epoch % 2 == 0:
        save_image(gen_imgs.data[:25], f"images/{epoch}_DCGAN_flowers2_512.png", nrow=5, normalize=True)
        # Save the model
        torch.save(generator.state_dict(), f"saved_model/saved_model_dcgan_flower2_512_{epoch}.pth")

# Save model after last iter
save_image(gen_imgs.data[:25], f"images/{epoch}_DCGAN_flowers2_512.png", nrow=5, normalize=True)
# Save the model
torch.save(generator.state_dict(), f"saved_model/saved_model_dcgan_flower2_512_{EPOCHS}.pth")


[Epoch 0/100] [Batch 0/512] [D loss: 0.6856919527053833] [G loss: 27.00840950012207]
[Epoch 0/100] [Batch 1/512] [D loss: 0.007181746885180473] [G loss: 20.93548583984375]
[Epoch 0/100] [Batch 2/512] [D loss: 0.00010157356882700697] [G loss: 20.059616088867188]
[Epoch 0/100] [Batch 3/512] [D loss: 0.00015276079648174345] [G loss: 17.40479850769043]
[Epoch 0/100] [Batch 4/512] [D loss: 2.7923491870751604e-05] [G loss: 12.769508361816406]
[Epoch 0/100] [Batch 5/512] [D loss: 0.009468231350183487] [G loss: 24.167055130004883]
[Epoch 0/100] [Batch 6/512] [D loss: 0.0035445382818579674] [G loss: 25.056812286376953]
[Epoch 0/100] [Batch 7/512] [D loss: 0.0013334851246327162] [G loss: 23.70978355407715]
[Epoch 0/100] [Batch 8/512] [D loss: 0.0004327901697251946] [G loss: 22.12247085571289]
[Epoch 0/100] [Batch 9/512] [D loss: 6.069765004212968e-05] [G loss: 17.920753479003906]
[Epoch 0/100] [Batch 10/512] [D loss: 8.827311103232205e-05] [G loss: 12.435054779052734]
[Epoch 0/100] [Batch 11/512